In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"
cohort = "GSE131027"

# Input paths
in_trait_dir = "../../input/GEO/Esophageal_Cancer"
in_cohort_dir = "../../input/GEO/Esophageal_Cancer/GSE131027"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/GSE131027.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/GSE131027.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/GSE131027.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# Based on the background information, the dataset contains germline variants and expression features
# related to mutations in cancer patients, so it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Looking at the Sample Characteristics Dictionary
# For trait (Esophageal_Cancer):
# Key 1 contains 'cancer: Oesophageal cancer', which indicates presence of trait data
trait_row = 1  # This contains cancer types including esophageal cancer

# For age:
# Age data is not available in the sample characteristics
age_row = None

# For gender:
# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert cancer type to binary (1 for Esophageal Cancer, 0 for other types)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        cancer_type = value.split(":", 1)[1].strip().lower()
    else:
        cancer_type = value.strip().lower()
    
    # Look for esophageal cancer with variation in spelling
    if "oesophageal" in cancer_type or "esophageal" in cancer_type:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age to continuous numeric value"""
    # Not used as age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    # Not used as gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering metadata
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    # Load the clinical data
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Clinical Features Preview:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_path}")


Clinical data file not found at ../../input/GEO/Esophageal_Cancer/GSE131027/clinical_data.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 69
Header line: "ID_REF"	"GSM3759992"	"GSM3759993"	"GSM3759994"	"GSM3759995"	"GSM3759996"	"GSM3759997"	"GSM3759998"	"GSM3759999"	"GSM3760000"	"GSM3760001"	"GSM3760002"	"GSM3760003"	"GSM3760004"	"GSM3760005"	"GSM3760006"	"GSM3760007"	"GSM3760008"	"GSM3760009"	"GSM3760010"	"GSM3760011"	"GSM3760012"	"GSM3760013"	"GSM3760014"	"GSM3760015"	"GSM3760016"	"GSM3760017"	"GSM3760018"	"GSM3760019"	"GSM3760020"	"GSM3760021"	"GSM3760022"	"GSM3760023"	"GSM3760024"	"GSM3760025"	"GSM3760026"	"GSM3760027"	"GSM3760028"	"GSM3760029"	"GSM3760030"	"GSM3760031"	"GSM3760032"	"GSM3760033"	"GSM3760034"	"GSM3760035"	"GSM3760036"	"GSM3760037"	"GSM3760038"	"GSM3760039"	"GSM3760040"	"GSM3760041"	"GSM3760042"	"GSM3760043"	"GSM3760044"	"GSM3760045"	"GSM3760046"	"GSM3760047"	"GSM3760048"	"GSM3760049"	"GSM3760050"	"GSM3760051"	"GSM3760052"	"GSM3760053"	"GSM3760054"	"GSM3760055"	"GSM3760056"	"GSM3760057"	"GSM3760058"	"GSM3760059"	"GSM3760060"	"GSM3760061"	"GSM3760062"	"GSM3760063"	"GSM3760064"	

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the dataset
# The identifiers like "1007_s_at", "1053_at", etc. appear to be Affymetrix probe IDs
# from a microarray platform, not standard human gene symbols.
# These probe IDs will need to be mapped to human gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE131027
Line 6: !Series_title = High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers
Line 7: !Series_geo_accession = GSE131027
Line 8: !Series_status = Public on May 11 2019
Line 9: !Series_submission_date = May 10 2019
Line 10: !Series_last_update_date = Jul 09 2019
Line 11: !Series_pubmed_id = 31263571
Line 12: !Series_summary = We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In 


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols
# From the gene annotation preview, we can see:
# - 'ID' column contains probe IDs like "1007_s_at" which match gene expression data's index
# - 'Gene Symbol' column contains the human gene symbols we need

# 2. Get a gene mapping dataframe with the relevant columns
prob_col = 'ID'
gene_col = 'Gene Symbol'

# Extract the mapping from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# Save the gene data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
First few rows of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression dataframe shape after mapping: (21278, 92)
First few rows of gene expression data:
          GSM3759992  GSM3759993  GSM3759994  GSM3759995  GSM3759996  \
Gene                                                                   
A1BG        4.390919    9.637094    5.370776    7.376019    9.747455   
A1BG-AS1    4.498580    4.911001    4.409248    4.958840    4.126732   
A1CF        7.712909   17.768014    8.704946   14.905013   16.923252   
A2M        14.491904   16.222561   15.166473   15.598188   15.317525   
A2M-AS1     6.186831    4.286041    5.067774    5.807062    3.963854   

          GSM3759997  GSM3759998  GSM3759999  GSM3760000  GSM3760001  ...  \
Gene                                                                  ...   
A1BG        7.568074   12.627785   12.227179    7.042437    5.118175  ...   
A1BG-AS1    5.894118    4.571268    4.925717    4.390274    4.578439  ...   
A1CF        7.351392   21.828093   20.830584   17.073983    8.206698  ...   
A2M        

Gene expression data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE131027.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (19845, 92)
First few genes with their expression values after normalization:
          GSM3759992  GSM3759993  GSM3759994  GSM3759995  GSM3759996  \
Gene                                                                   
A1BG        4.390919    9.637094    5.370776    7.376019    9.747455   
A1BG-AS1    4.498580    4.911001    4.409248    4.958840    4.126732   
A1CF        7.712909   17.768014    8.704946   14.905013   16.923252   
A2M        14.491904   16.222561   15.166473   15.598188   15.317525   
A2M-AS1     6.186831    4.286041    5.067774    5.807062    3.963854   

          GSM3759997  GSM3759998  GSM3759999  GSM3760000  GSM3760001  ...  \
Gene                                                                  ...   
A1BG        7.568074   12.627785   12.227179    7.042437    5.118175  ...   
A1BG-AS1    5.894118    4.571268    4.925717    4.390274    4.578439  ...   
A1CF        7.351392   21.828093   20.830584   17.073983    8.206698  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE131027.csv


Raw clinical data shape: (5, 93)
Clinical features:
                   GSM3759992  GSM3759993  GSM3759994  GSM3759995  GSM3759996  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   

                   GSM3759997  GSM3759998  GSM3759999  GSM3760000  GSM3760001  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   

                   ...  GSM3760074  GSM3760075  GSM3760076  GSM3760077  \
Esophageal_Cancer  ...         0.0         0.0         0.0         0.0   

                   GSM3760078  GSM3760079  GSM3760080  GSM3760081  GSM3760082  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   

                   GSM3760083  
Esophageal_Cancer         0.0  

[1 rows x 92 columns]
Clinical features saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE131027.csv
Linked data shape: (92, 19846)
Linked data preview (first 5 rows, first 5 columns):
            Esophageal_Cancer      A1BG  A

Data shape after handling missing values: (92, 19846)
For the feature 'Esophageal_Cancer', the least common label is '1.0' with 1 occurrences. This represents 1.09% of the dataset.
The distribution of the feature 'Esophageal_Cancer' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
